In [1]:
import pickle
import gzip
import pandas as pd
import numpy as np
review = pd.read_json('/users/tr.lisusi/yelp_dataset/round13/review.json', lines=True)
print(len(review))
print(list(review))

6685900
['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text', 'useful', 'user_id']


In [2]:
user_item_date = review.drop(['cool','funny','review_id','stars','text','useful'],axis = 1)
del review
from collections import defaultdict
from itertools import count
uid_map = defaultdict(count().__next__)
iid_map = defaultdict(count().__next__)
uids = np.array([uid_map[uid] for uid in user_item_date["user_id"].values ], dtype=np.int32)
iids = np.array([iid_map[iid] for iid in user_item_date["business_id"].values ], dtype=np.int32)

uid_rev_map = {v: k for k, v in uid_map.items()}
iid_rev_map = {v: k for k, v in iid_map.items()}

def digital(data):
    data['business_id'] = iid_map[data['business_id']]
    data['user_id'] = uid_map[data['user_id']]
    return data

user_item_date = user_item_date.apply(digital,axis=1)

In [3]:
"""remove all venues with less than 10 checkins"""
lesscheckin_filter = user_item_date.groupby('business_id').filter(lambda x: len(x)>=10)
"""ground truth for all users"""
ground = lesscheckin_filter.sort_values('date').groupby('user_id').tail(1)
"""remove all checkin from ground truth since many users have several checkins on ground truth venue"""
counter = 0
index = []
all_user = list(ground['user_id'])
for u in all_user:
    ground_venue = float(ground.loc[ground['user_id']==u]['business_id'])
    other_venues = list(lesscheckin_filter.loc[lesscheckin_filter['user_id']==u]['business_id'])
    if ground_venue in other_venues:
        index.extend(list(lesscheckin_filter.loc[(lesscheckin_filter['user_id']==u) 
                                              & (lesscheckin_filter['business_id']==ground_venue)].index.values))
    
    counter+=1
    if counter%100000==0:print(counter)


print(len(index))
print(len(lesscheckin_filter))
new_dataset = lesscheckin_filter.drop(index)
print(len(new_dataset))

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1584133
6199717
4615584


In [4]:
from spotlight.interactions import Interactions
import torch
from spotlight.factorization.implicit import ImplicitFactorizationModel
import time
from spotlight.losses import regression_loss
from spotlight.torch_utils import cpu, gpu, minibatch, set_seed, shuffle
from spotlight.losses import bpr_loss
from spotlight.sampling import sample_items
from spotlight.factorization.implicit import ImplicitFactorizationModel
train = Interactions(user_ids=uids, item_ids=iids)
uids = np.array(new_dataset["user_id"].values , dtype=np.int32)
iids = np.array(new_dataset["business_id"].values , dtype=np.int32)

In [5]:
model = ImplicitFactorizationModel(n_iter=10,embedding_dim=10,use_cuda=True, loss = 'bpr')

current = time.time()
model.fit(train, verbose=True)

end = time.time()
diff = end - current
print("Training took %d seconds, gpu=%s "% (diff, torch.cuda.is_available()))

Epoch 0: loss 0.18935772559279646
Epoch 1: loss 0.07945726898788354
Epoch 2: loss 0.04951970218730901
Epoch 3: loss 0.03778654812117079
Epoch 4: loss 0.03213484586299703
Epoch 5: loss 0.028974542955152935
Epoch 6: loss 0.026932720234597977
Epoch 7: loss 0.025483417464757182
Epoch 8: loss 0.024291944306039048
Epoch 9: loss 0.023314035158883882
Training took 1767 seconds, gpu=True 


In [7]:
from random import sample
unique_user = list(ground['user_id'])
print(len(unique_user))
unique_business = list(np.arange(max(new_dataset['business_id'])+1))
testset = {}
for user in unique_user:
    
    visited_venues = list(new_dataset.loc[new_dataset['user_id']==user,'business_id'])
    unvisited_venues = sample(unique_business,100)
    
    while len(set(unvisited_venues)&set(visited_venues))!=0:
        unvisited_venues = sample(unique_business,100)
        
    testset[user] = unvisited_venues
    if len(testset)%100000 ==0: print(len(testset))

1543938
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000


In [8]:
"""reconstruct ground as dict"""
adder=0
dict_ground = {}
for index, row in ground.iterrows():
    dict_ground[row['user_id']] = row['business_id']
    adder+=1
    if adder%100000==0:print(adder)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000


In [10]:
import time
def position_finder (predictions,groundtruth):
    position = 1
    for i in predictions:
        if groundtruth < i:
            position+=1
        else:
            break
    return position

    
def ndcg_calculator(user_id , model,bound,ground):
    ndcg = 0
    rel = 1
    position = 0
    unvisited_predictions = np.sort(model.predict(user_id)[testset[user_id]])[::-1][0:10]
    ground_venue = int(ground[user_id])
    if ground_venue <= bound:
        ground_prediction = model.predict(user_id)[ground_venue]
        if ground_prediction >= min(unvisited_predictions):
            position = position_finder(unvisited_predictions,ground_prediction)
            ndcg = rel/(np.log2(position + 1))
        else:
            ndcg = 0
    else:
        ndcg = 0
    return ndcg
adder = 0
NDCG = []
print(len(testset.keys()))
max_business = max(iids)
max_user = max(uids)
for u in list(testset.keys()):
    if u <=max_user:
        NDCG.append(ndcg_calculator(u,model,max_business,dict_ground))
    adder+=1
    if adder%100000==0:
        print(adder)
        print(np.array(NDCG).mean())
print(np.array(NDCG).mean())

1543938
100000
0.8270928630655703
200000
0.8206537771145014
300000
0.8185957975306104
400000
0.8171492892529542
500000
0.8155773632541821
600000
0.8136716741519692
700000
0.811778670037556
800000
0.8096782874107062
900000
0.807481696896387
1000000
0.8051780380493313
1100000
0.8026126179537212
1200000
0.799910775769902
1300000
0.7969237701443292
1400000
0.7938629677780756
1500000
0.7898267901322226
0.7879669573470498
